In [1]:
import lumipy as lm
import datetime as dt

In [2]:
# Get the atlas again up here
atlas = lm.get_atlas(api_secrets_filename='secrets.json')

Getting atlas🌎
  • Querying data provider metadata...
  • Querying direct provider metadata...
  • Building atlas...
Done!
Contents: 
  • 273 data providers
  • 18 direct providers


# Query Scripting I

The other role of the `atlas` is as the starting point for `lumipy`'s fluent query syntax. You start by initialising a table object from an atlas attribute. 
```
    table = atlas.my_provider()
```

If the provider takes parameters they are specified here as keyword arguments, not in the `where` clause. This is different to normal luminesce SQL. 
```
    table = atlas.my_provider(param1=value1)
```

## Query 1A: Select and Limit

### Background
Once the table object is made, the next step is to chain methods, beginning with `.select`, to build up the query. 
```
    query = table.select('*')
```

Finally, once that's done and you're ready to send the query you just call `.go()` on the query object.

```
    df = query.go()
```

### Exercise
Select the main columns (`'^'`) limited to 10 rows from the `lusid_portfolio` provider with an `as_at` parameter equal to 2022/09/01. Use a python `datetime` object as the value: `dt.datetime(2022, 9, 1)`.

In [3]:
pf = atlas.lusid_portfolio(as_at=dt.datetime(2022, 9, 1))
pf.select('^').limit(10).go()

,BaseCurrency,PortfolioCode,PortfolioScope,PortfolioType
0,USD,357090_1001,POINT,Transaction
1,USD,357070_1003,POINT,Transaction
2,USD,357090_1002,POINT,Transaction
3,USD,357070_1001,POINT,Transaction
4,USD,357050_1001,POINT,Transaction
5,USD,358780_1001,POINT,Transaction
6,USD,358630_1001,POINT,Transaction
7,USD,356870_1001,POINT,Transaction
8,GBP,pension-fund-8244,production-client-377e-cf72-4662-04,Transaction
9,USD,356870_1002,POINT,Transaction


## Query 1B: New Columns

### Background
New columns of values are built up by applying python operations to table columns. These columns live as attributes on the table objects, for example
```
    (table.value1 + table.value1) / 2
```

To use these in a query you supply them to the `.select` method as keyword argyments. The keyword will be the name granted to the column

```
    query = table.select('*', Doubled=table.col * 2)
```

### Exercise
Create a new column called `LoudNoises` that is the portfolio code string converted to upper case. You can do this similarly to pandas by using the `.str` accessor and the associated method
```
    table.column.str.upper()
```

In [4]:
pf.select('^', LoudNoises=pf.portfolio_code.str.upper()).limit(10).go()

,BaseCurrency,PortfolioCode,PortfolioScope,PortfolioType,LoudNoises
0,USD,358630_1001,POINT,Transaction,358630_1001
1,USD,356870_1002,POINT,Transaction,356870_1002
2,USD,357090_1002,POINT,Transaction,357090_1002
3,USD,357090_1001,POINT,Transaction,357090_1001
4,USD,358780_1001,POINT,Transaction,358780_1001
5,USD,356870_1001,POINT,Transaction,356870_1001
6,USD,357050_1001,POINT,Transaction,357050_1001
7,USD,357070_1001,POINT,Transaction,357070_1001
8,GBP,pension-fund-8244,production-client-377e-cf72-4662-04,Transaction,PENSION-FUND-8244
9,USD,357070_1003,POINT,Transaction,357070_1003


## Query 2: Where 

### Background
Other methods are chained in the standard SQL order of `select`, `where`, `group_by`, `aggregate`, `having`, `order_by` then `limit`. 

Let's try filtering with a where clause next. Where clauses are added by chaining `.where` after `.select` and giving a column expression that resolves to a boolean as its argument. 
```
    table.select('*').where(table.col == value)
```

### Exercise
Now filter for the portfolios that have a `portfolio_scope` equal to the `Finbourne-Examples` scope. Use the `example_scope` variable below. 

In [5]:
example_scope = 'Finbourne-Examples'

In [6]:
pf.select('^').where(pf.portfolio_scope == example_scope).go()

,BaseCurrency,PortfolioCode,PortfolioScope,PortfolioType
0,USD,US-Futures,Finbourne-Examples,Transaction
1,USD,US-Treasury-Bond,Finbourne-Examples,Transaction
2,EUR,Global-Equity-swap,Finbourne-Examples,Transaction
3,GBP,UK-LegSwaps,Finbourne-Examples,Transaction
4,GBP,Global-Futures,Finbourne-Examples,Transaction
5,GBP,UK-Equities,Finbourne-Examples,Transaction
6,GBP,Global-Equity,Finbourne-Examples,Transaction
7,GBP,UK-Swaps,Finbourne-Examples,Transaction
8,USD,US-Corporate-Bond,Finbourne-Examples,Transaction
9,USD,US-Futures1,Finbourne-Examples,Transaction


## Case Statements and Printing SQL

### Background
What's happening as you operate on and call methods on columns is that you're building up an a graph that represent bits of SQL. You can see the SQL that it resolves to with `.print_sql()`.

### Exercise
Print the underlying SQL for the region case statement that's constructed below by calling `.print_sql()`and (optionally) print the graph by just printing the variable `region`

In [8]:
region = pf.when(
    pf.portfolio_code.str.contains('global')
).then(
    "Global"
).when(
    pf.portfolio_code.str.contains('US')
).then(
    'USA'
).when(
    pf.portfolio_code.str.contains('UK')
).then(
    'UK'
).otherwise(
    'OTHER'
)

In [9]:
region.print_sql()

(  
  CASE
      WHEN [PortfolioCode] LIKE '%global%'
        THEN 'Global'
      WHEN [PortfolioCode] LIKE '%US%'
        THEN 'USA'
      WHEN [PortfolioCode] LIKE '%UK%'
        THEN 'UK'
    ELSE 'OTHER'
  END
)


## Query 3: Grouping and Aggregating

### Background 

Grouping and aggregation in `lumipy` works similarly to `pandas`. In `pandas` you'd have something like this 
```
    df.groupby('Category').agg(ColumnMean=('Column', 'mean'))
```
it's structured in a similar way in `lumipy`. Here you also `group_by` and `aggregate` as separate methods and give values to the latter as keyword arguments. One difference in this case is that the aggregation function live as methods on the columns
```
    table.select(
        table.category
    ).group_by(
        table.category
    ).aggregate(
        ColumnMean=table.column.mean()
    )
```


### Exercise
Group the table by the region varible (the switch statement above) and count how many portfolios there are in each region in the `Finbourne-Examples` scope. Call this new column `PortfolioCount`. Use `.count()` in the `.aggregate()` method.

In [10]:
pf.select(
    Region=region
).where(
    (pf.portfolio_scope == example_scope)
).group_by(
    region
).aggregate(
    PortfolioCount=pf.portfolio_code.count()
).go()

,Region,PortfolioCount
0,Global,3
1,OTHER,6
2,UK,3
3,USA,4
